In [5]:
import pandas as senju
import numpy as nandini
import matplotlib.pyplot as kichu 
from wordcloud import WordCloud as dhamo
from gensim.models import Word2Vec

In [6]:
df = senju.read_csv('twt_data.csv')
df.sample(4)

,TWEET,Subjectivity,Polarity,Score
356,First BJP leader talked sensibly NupurSharma....,0.333333,0.250000,Positive
468,"BJP, Congress Create Ruckus Over Growing Unemp...",0.000000,0.000000,Neutral
121,The service youre doing.. big salute you ma...,0.255556,0.166667,Positive
102,"BJP dynastic party, they are all sons Bha...",0.000000,0.000000,Neutral


In [7]:
df['Score'].value_counts()/df.shape[0]

Neutral     0.506
Positive    0.304
Negative    0.190
Name: Score, dtype: float64

In [8]:
y = senju.get_dummies(df['Score'])

In [9]:
y

,Negative,Neutral,Positive
0,0,1,0
1,1,0,0
2,0,1,0
3,1,0,0
4,0,1,0
...,...,...,...
495,0,1,0
496,0,1,0
497,1,0,0
498,0,1,0


In [10]:
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import wordnet
from nltk.corpus import stopwords
stopwords.words('english')
import nltk
nltk.download('all')
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\NANDHINI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NANDHINI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\NANDHINI\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\NANDHINI\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\NANDHINI\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_per

In [11]:
#%%capture
class NLP_Preprocessor:
    def __init__(self):
        pass

    def normalize_text(self,text):
        text = text.lower()
        text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(pic\.twitter\.com/[^\s]+))','', text)
        text = re.sub('@[^\s]+','', text)
        text = re.sub('#([^\s]+)', '', text)
        text = re.sub('[:;>?<=*+()&,\-#!$%\{˜|\}\[^_\\@\]1234567890’‘]',' ', text)
        text = re.sub('[\d]','', text)
        text = text.replace(".", '')
        text = text.replace("'", '')
        text = text.replace("`", '')
        text = text.replace("'s", '')
        text = text.replace("/", ' ')
        text = text.replace("\"", ' ')
        text = text.replace("\\", '')
        #text =  re.sub(r"\b[a-z]\b", "", text)
        text=re.sub( '\s+', ' ', text).strip()
        return text
    
    def normalize_tokens(self, datacolumn):
        return datacolumn.apply(self.normalize_text)

    def remove_stopwords(self,input_text):
        stopwords_list = stopwords.words('english')
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def remove_stopwords_tokens(self, tokens):
        return tokens.apply(self.remove_stopwords)
    
    def Lemmatize(self,text):
        lemmatizer = WordNetLemmatizer()
        lemmatizer.lemmatize(text)
        return lemmatizer.lemmatize(text,pos = 'v')
    
    def apply_lemmatize(self,tokens):
        return tokens.apply(self.Lemmatize)
    
    def tokenize(self, datacolumn):
        return [[nltk.word_tokenize(token)] for token in datacolumn]
    
    def word2tokens(self,text):
        return [word for word in text.split()]
    
    def toword2vec(self, datacolumn):
        return datacolumn.apply(self.word2tokens)
    
    def preprocess(self, datacolumn):
        tokens = self.normalize_tokens(datacolumn)
        tokens = self.remove_stopwords_tokens(tokens)
        tokens = self.apply_lemmatize(tokens)
        tokens = self.toword2vec(tokens)
        # tokens = self.tokenize(tokens)
        return tokens


In [12]:
nlp = NLP_Preprocessor()

In [13]:
bfr = nlp.preprocess(df['TWEET'])

In [14]:
len(bfr)

500

In [15]:
bfr[0]

['delhi',
 'bjp',
 'national',
 'president',
 'jpnadda',
 'pays',
 'tribute',
 'bharatiya',
 'jan',
 'sangh',
 'founder',
 'syama',
 'prasad',
 'mookerjee',
 'part…']

In [16]:
X = bfr

In [17]:
senju.DataFrame(X).sample(3)

,TWEET
117,"[howrah, railway, station, shri, syamaprasadmo..."
30,"[wonderful, photo, 🇨🇮, bjps, presidential, can..."
258,"[bias, judges, give, right, judgments, review,..."


In [18]:
model = Word2Vec(X, min_count=1)

In [19]:
model.build_vocab(X)

In [20]:
model.wv.vector_size

100

In [21]:
model.wv.most_similar('bjp')

[('indianmedia', 0.38324207067489624),
 ('letting', 0.35042691230773926),
 ('modis', 0.3431374132633209),
 ('review', 0.31404203176498413),
 ('dictatorship', 0.31053704023361206),
 ('elaborates', 0.30851539969444275),
 ('thesilentcoup', 0.2957453429698944),
 ('ipc', 0.2910431921482086),
 ('may', 0.2776744067668915),
 ('part', 0.27676475048065186)]

TFIDF

In [34]:
def to_text(list):
    return ' '.join([str(i) for i in list])

In [36]:
X = X.apply(to_text)

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer as priyanga
tf = priyanga(max_features=1500)

In [42]:
from sklearn.model_selection import train_test_split as tts
x_train, x_test, y_train, y_test = tts(X, y, test_size=0.2, random_state=0)

In [43]:
tf.fit(x_train)

TfidfVectorizer(max_features=1500)

In [55]:
tf.transform(x_train).toarray()[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [56]:
from tqdm import tqdm
def check_tfidf(array):
    cnt = 0
    for i in tqdm(array):
        if i!= 0:
            cnt += 1
    return cnt

check_tfidf(tf.transform(x_train).toarray()[0])

100%|██████████| 1500/1500 [00:00<00:00, 1505853.52it/s]


8

In [57]:
x_train = tf.transform(x_train).toarray()
x_test = tf.transform(x_test).toarray()

Logistic regression

In [58]:
from sklearn.linear_model import LogisticRegression as lor
from sklearn.multiclass import OneVsRestClassifier as orc

lr = lor()
ovr = orc(lr)
ovr.fit(x_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression())

In [59]:
predict = ovr.predict(x_test)

In [60]:
predict

array([[0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0,

Naive Bayes classifier

In [61]:
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
oer = orc(nb)
oer.fit(x_train, y_train)

OneVsRestClassifier(estimator=GaussianNB())

In [63]:
y_test

,Negative,Neutral,Positive
90,0,1,0
254,0,1,0
283,0,1,0
445,1,0,0
461,0,0,1
...,...,...,...
372,0,0,1
56,0,1,0
440,0,1,0
60,0,0,1


In [62]:
oer.predict(x_test)

array([[0, 1, 0],
       [0, 1, 1],
       [0, 0, 0],
       [0, 0, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 0, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 1, 0],
       [0, 1, 0],
       [0, 0, 1],
       [0, 1, 0],
       [1, 1, 1],
       [0, 0, 1],
       [0, 0, 0],
       [0, 1, 0],
       [0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [0, 1, 0],
       [1, 1, 0],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 1],
       [0, 1, 0],
       [0, 1, 0],
       [1, 0, 0],
       [0, 0, 1],
       [0, 0, 1],
       [0,